# 00 — Sanity Check

This notebook validates the **Milestone 1** functionality of **Crypto Project Risk Radar**:

- Loads sample project inputs (`data/sample/projects.json`)
- Scores each project using the deterministic rubric
- Writes machine-readable outputs to `results/reports.json`
- Performs a few basic invariants (score bounds, determinism)

> **Note:** This project is for educational research only and is **not financial advice**.


## 1) Environment + path setup

This notebook assumes it is run from the repository root (or can find it via relative paths).


In [1]:
from pathlib import Path
import json

# Try to locate repo root from this notebook location
NOTEBOOK_PATH = Path.cwd()

# If running from repo root, notebooks/ exists below.
# If running from within notebooks/, root is parent.
if (NOTEBOOK_PATH / "notebooks").exists():
    REPO_ROOT = NOTEBOOK_PATH
elif NOTEBOOK_PATH.name == "notebooks" and (NOTEBOOK_PATH.parent / "pyproject.toml").exists():
    REPO_ROOT = NOTEBOOK_PATH.parent
else:
    # fallback: walk upward until pyproject.toml
    REPO_ROOT = None
    for p in [NOTEBOOK_PATH] + list(NOTEBOOK_PATH.parents):
        if (p / "pyproject.toml").exists():
            REPO_ROOT = p
            break
    if REPO_ROOT is None:
        raise RuntimeError("Could not locate repo root (pyproject.toml not found).")

DATA_SAMPLE = REPO_ROOT / "data" / "sample" / "projects.json"
RESULTS_DIR = REPO_ROOT / "results"
OUTPUT_JSON = RESULTS_DIR / "reports.json"

print("Repo root:", REPO_ROOT)
print("Sample input:", DATA_SAMPLE)
print("Results dir:", RESULTS_DIR)
print("Output path:", OUTPUT_JSON)

assert DATA_SAMPLE.exists(), f"Missing sample input file: {DATA_SAMPLE}"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)


Repo root: c:\Projects\crypto-project-risk-radar
Sample input: c:\Projects\crypto-project-risk-radar\data\sample\projects.json
Results dir: c:\Projects\crypto-project-risk-radar\results
Output path: c:\Projects\crypto-project-risk-radar\results\reports.json


## 2) Import project code

If you installed the package in editable mode (`pip install -e .`), imports should work immediately.
If you haven't installed yet, run the install command in your terminal.


In [2]:
from crypto_risk_radar.io import load_projects_json
from crypto_risk_radar.rubric import score_project
from crypto_risk_radar.report import save_reports_json

print("Imports OK")


Imports OK


## 3) Load sample projects


In [3]:
projects = load_projects_json(DATA_SAMPLE)
print(f"Loaded {len(projects)} projects")
projects[:2]


Loaded 2 projects


[Project(project_id='demo-001', name='ExampleSwap', chain='Ethereum', category='DeFi', website='https://exampleswap.invalid', docs_url='https://docs.exampleswap.invalid', repo_url='https://github.com/example/exampleswap', launch_date='2025-01-15', has_whitepaper=True, has_public_repo=True, team_identified=True, audit_reported=False, tvl_usd=None, fdv_usd=None, volume_24h_usd=None),
 Project(project_id='demo-002', name='MysteryCoin', chain='', category='Meme', website=None, docs_url=None, repo_url=None, launch_date=None, has_whitepaper=False, has_public_repo=False, team_identified=False, audit_reported=False, tvl_usd=None, fdv_usd=None, volume_24h_usd=None)]

## 4) Score projects + basic invariants

Milestone 1 is deterministic: same input should always yield the same output.


In [4]:
reports_1 = [score_project(p) for p in projects]
reports_2 = [score_project(p) for p in projects]

# Invariants
for r in reports_1:
    assert 0.0 <= r.overall_risk_score <= 100.0, f"Score out of bounds: {r}"

# Determinism check (overall score + breakdown keys)
for a, b in zip(reports_1, reports_2):
    assert a.overall_risk_score == b.overall_risk_score
    assert a.risk_breakdown == b.risk_breakdown

print("Scoring OK ✅  (bounds + determinism checks passed)")
reports_1


Scoring OK ✅  (bounds + determinism checks passed)


[RiskReport(project_id='demo-001', name='ExampleSwap', overall_risk_score=15.0, risk_breakdown={'no_audit': 15.0}, notes=['No audit reported.']),
 RiskReport(project_id='demo-002', name='MysteryCoin', overall_risk_score=100.0, risk_breakdown={'no_whitepaper': 20.0, 'no_public_repo': 20.0, 'anonymous_team': 15.0, 'no_audit': 15.0, 'unknown_chain_or_category': 10.0, 'missing_links': 10.0, 'very_new_project': 10.0}, notes=['No whitepaper indicated.', 'No public code repository indicated.', 'Team not identified.', 'No audit reported.', 'Chain and/or category missing.', 'Website and/or documentation link missing.', 'Launch date missing; treating as higher uncertainty.'])]

## 5) Save results artifact

This writes a machine-readable output file that later milestones will extend.


In [5]:
save_reports_json(reports_1, OUTPUT_JSON)
print("Wrote:", OUTPUT_JSON)
assert OUTPUT_JSON.exists(), "Output JSON file was not created"
print("Output size (bytes):", OUTPUT_JSON.stat().st_size)


Wrote: c:\Projects\crypto-project-risk-radar\results\reports.json
Output size (bytes): 893


## 6) Read back output and validate shape


In [6]:
payload = json.loads(OUTPUT_JSON.read_text(encoding="utf-8"))
assert isinstance(payload, list)
assert len(payload) == len(projects)

required_keys = {"project_id", "name", "overall_risk_score", "risk_breakdown", "notes"}
for item in payload:
    assert required_keys.issubset(item.keys())
    assert 0.0 <= float(item["overall_risk_score"]) <= 100.0

payload


[{'project_id': 'demo-001',
  'name': 'ExampleSwap',
  'overall_risk_score': 15.0,
  'risk_breakdown': {'no_audit': 15.0},
  'notes': ['No audit reported.']},
 {'project_id': 'demo-002',
  'name': 'MysteryCoin',
  'overall_risk_score': 100.0,
  'risk_breakdown': {'no_whitepaper': 20.0,
   'no_public_repo': 20.0,
   'anonymous_team': 15.0,
   'no_audit': 15.0,
   'unknown_chain_or_category': 10.0,
   'missing_links': 10.0,
   'very_new_project': 10.0},
  'notes': ['No whitepaper indicated.',
   'No public code repository indicated.',
   'Team not identified.',
   'No audit reported.',
   'Chain and/or category missing.',
   'Website and/or documentation link missing.',
   'Launch date missing; treating as higher uncertainty.']}]

## 7) Optional: run the CLI end-to-end

This is optional, but it validates the packaging + entry workflow.

If this fails due to environment configuration, that’s OK — Milestone 1 is still valid as long as imports and scoring work.


In [7]:
import subprocess
import sys

cmd = [sys.executable, "-m", "crypto_risk_radar.cli", "--input", str(DATA_SAMPLE), "--output", str(OUTPUT_JSON)]
print("Running:", " ".join(cmd))
result = subprocess.run(cmd, cwd=str(REPO_ROOT), capture_output=True, text=True)

print("Return code:", result.returncode)
print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)

assert result.returncode == 0, "CLI execution failed"
print("CLI OK ✅")


Running: c:\Projects\crypto-project-risk-radar\.venv\Scripts\python.exe -m crypto_risk_radar.cli --input c:\Projects\crypto-project-risk-radar\data\sample\projects.json --output c:\Projects\crypto-project-risk-radar\results\reports.json
Return code: 0
STDOUT:
 Wrote 2 reports to c:\Projects\crypto-project-risk-radar\results\reports.json

STDERR:
 
CLI OK ✅


## Done

If everything above ran without errors, Milestone 1 is operational:
- deterministic rubric works
- sample data loads
- outputs are produced
